In [ ]:
import pandas as pd

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForPreTraining



In [ ]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert_large")
model = AutoModelForPreTraining.from_pretrained("csebuetnlp/banglabert_large")

In [ ]:
def convert_text_to_tuple_from_csv(file_path):
      data = []
      df = pd.read_csv(file_path)
      x = 'Sentence: 1'
      temp = []
      for index, row in df.iterrows():
        flag = row['Sentence #']
        if flag == x:
          temp.append((row["Word"], row["Tag"]))
          # print (index,row['Sentence #'],row["Word"], row["Tag"])
        else:
          x = row['Sentence #']
          data.append(temp)
          temp = [(row["Word"], row["Tag"])]

      return data
dev = convert_text_to_tuple_from_csv('/content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/dev2022withPos.csv')
train = convert_text_to_tuple_from_csv('/content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/train2022withPos.csv')

In [ ]:
def convert_text_to_tuple_from_csv1(file_path):
      data = []
      df = pd.read_csv(file_path)
      x = 'Sentence: 1'
      temp = []
      for index, row in df.iterrows():
        flag = row['Sentence #']
        if flag == x:
          temp.append((row["Word"], row["Tag"]))
          # print (index,row['Sentence #'],row["Word"], row["Tag"])
        else:
          x = row['Sentence #']
          data.append(temp)
          temp = [(row["Word"], row["Tag"])]

      return data[:10000]

test =  convert_text_to_tuple_from_csv1('/content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/test2022withPos.csv')

In [ ]:
len(test)

10000

In [ ]:
class NERDataMaker:
    def __init__(self, texts):
        self.unique_entities = ['O', 'B-CORP', 'I-CORP', 'B-CW', 'I-CW', 'B-GRP', 'I-GRP', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD']
        self.processed_texts = []

        temp_processed_texts = []
        for text in texts:
            tokens_with_entities = text
            for _, ent in tokens_with_entities:
                if ent not in self.unique_entities:
                    self.unique_entities.append(ent)
            temp_processed_texts.append(tokens_with_entities)

        # self.unique_entities.sort(key=lambda ent: ent if ent != "O" else "")
        print(self.unique_entities)
        for tokens_with_entities in temp_processed_texts:
            self.processed_texts.append([(t, self.unique_entities.index(ent)) for t, ent in tokens_with_entities])

    @property
    def id2label(self):
        return {0: 'O', 1: 'B-CORP', 2: 'I-CORP', 3: 'B-CW', 4: 'I-CW', 5: 'B-GRP', 6: 'I-GRP', 7: 'B-LOC', 8: 'I-LOC', 9: 'B-PER', 10: 'I-PER', 11: 'B-PROD', 12: 'I-PROD'}

    @property
    def label2id(self):
        return {'O': 0, 'B-CORP': 1, 'I-CORP': 2, 'B-CW': 3, 'I-CW': 4, 'B-GRP': 5, 'I-GRP': 6, 'B-LOC': 7, 'I-LOC': 8, 'B-PER': 9, 'I-PER': 10, 'B-PROD': 11, 'I-PROD': 12}

    def __len__(self):
        return len(self.processed_texts)

    def __getitem__(self, idx):
        def _process_tokens_for_one_text(id, tokens_with_encoded_entities):
            ner_tags = []
            tokens = []
            for t, ent in tokens_with_encoded_entities:
                ner_tags.append(ent)
                tokens.append(t)

            return {
                "id": id,
                "ner_tags": ner_tags,
                "tokens": tokens
            }

        tokens_with_encoded_entities = self.processed_texts[idx]
        if isinstance(idx, int):
            return _process_tokens_for_one_text(idx, tokens_with_encoded_entities)
        else:
            return [_process_tokens_for_one_text(i+idx.start, tee) for i, tee in enumerate(tokens_with_encoded_entities)]

    def as_hf_dataset(self, tokenizer):
        from datasets import Dataset, Features, Value, ClassLabel, Sequence
        def tokenize_and_align_labels(examples):
            tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

            labels = []
            for i, label in enumerate(examples[f"ner_tags"]):
                word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
                previous_word_idx = None
                label_ids = []
                for word_idx in word_ids:  # Set the special tokens to -100.
                    if word_idx is None:
                        label_ids.append(-100)
                    elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                        label_ids.append(label[word_idx])
                    else:
                        label_ids.append(-100)
                    previous_word_idx = word_idx
                labels.append(label_ids)

            tokenized_inputs["labels"] = labels
            return tokenized_inputs

        ids, ner_tags, tokens = [], [], []
        for i, pt in enumerate(self.processed_texts):
            ids.append(i)
            pt_tokens,pt_tags = list(zip(*pt))
            ner_tags.append(pt_tags)
            tokens.append(pt_tokens)
        data = {
            "id": ids,
            "ner_tags": ner_tags,
            "tokens": tokens
        }
        features = Features({
            "tokens": Sequence(Value("string")),
            "ner_tags": Sequence(ClassLabel(names=dm.unique_entities)),
            "id": Value("int32")
        })
        ds = Dataset.from_dict(data, features)
        tokenized_ds = ds.map(tokenize_and_align_labels, batched=True)
        return tokenized_ds

# usage
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert_large")
dm = NERDataMaker(dev)
dm1 = NERDataMaker(train) 

['O', 'B-CORP', 'I-CORP', 'B-CW', 'I-CW', 'B-GRP', 'I-GRP', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD']
['O', 'B-CORP', 'I-CORP', 'B-CW', 'I-CW', 'B-GRP', 'I-GRP', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD']


In [ ]:
dm2 = NERDataMaker(test)

['O', 'B-CORP', 'I-CORP', 'B-CW', 'I-CW', 'B-GRP', 'I-GRP', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD']


In [ ]:
dm1.id2label

{0: 'O',
 1: 'B-CORP',
 2: 'I-CORP',
 3: 'B-CW',
 4: 'I-CW',
 5: 'B-GRP',
 6: 'I-GRP',
 7: 'B-LOC',
 8: 'I-LOC',
 9: 'B-PER',
 10: 'I-PER',
 11: 'B-PROD',
 12: 'I-PROD'}

TypeError: ignored

In [ ]:
for i in dm:
  print(i)
  Dataset.from_dict(i)

{'id': 0, 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0], 'tokens': ['সমস্ত', 'বেতন', 'নিলামের', 'সাধারণ', 'ব্যবহারিক', 'উদাহরণ', 'বিভিন্ন', 'পেনি', 'নিলাম', '/', 'বিডিং', 'ফি', 'নিলাম', 'ওয়েবসাইটে', 'পাওয়া', 'যাবে']}


TypeError: ignored

In [ ]:
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
def tokenize_function(examples):
    return tokenizer(examples["tokens"], padding="max_length", truncation=True, is_split_into_words=False)

tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert_large")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenize_function)
model = AutoModelForTokenClassification.from_pretrained("csebuetnlp/banglabert_large", num_labels=len(dm1.unique_entities), id2label=dm1.id2label, label2id=dm1.label2id)

Some weights of the model checkpoint at csebuetnlp/banglabert_large were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at csebuetnlp/banglabert_large and are newly initialized: ['classifier.bias', 'classifier.weight

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
from datasets import load_metric
metric = load_metric("seqeval")

label_list = ['O', 'B-CORP', 'I-CORP', 'B-CW', 'I-CW', 'B-GRP', 'I-GRP', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD']
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    weight_decay=0.01,
)

train_ds = dm1.as_hf_dataset(tokenizer=tokenizer)
dev_ds = dm.as_hf_dataset(tokenizer=tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=dev_ds, # eval on training set! ONLY for DEMO!!
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Map:   0%|          | 0/15300 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


AttributeError: ignored

In [ ]:
model.get_input_embeddings()

Embedding(32000, 1024, padding_idx=0)

In [ ]:
test_ds = dm2.as_hf_dataset(tokenizer=tokenizer)

  0%|          | 0/133 [00:00<?, ?ba/s]

In [ ]:
trainer.evaluate(test_ds)

***** Running Evaluation *****
  Num examples = 132962
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `ElectraForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `ElectraForTokenClassification.forward`,  you can safely ignore this message.


{'eval_loss': 0.9631012678146362,
 'eval_precision': 0.6461371915092822,
 'eval_recall': 0.6521716660883432,
 'eval_f1': 0.6491404048309285,
 'eval_accuracy': 0.8652259269539472,
 'eval_runtime': 747.8335,
 'eval_samples_per_second': 177.796,
 'eval_steps_per_second': 2.779,
 'epoch': 20.0}

In [ ]:
pred = trainer.predict(test_ds)

***** Running Prediction *****
  Num examples = 1934
  Batch size = 64
The following columns in the test set don't have a corresponding argument in `ElectraForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `ElectraForTokenClassification.forward`,  you can safely ignore this message.


In [ ]:
type(pred)

transformers.trainer_utils.PredictionOutput

In [ ]:
trainer.save_model(output_dir = '/content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/model parameters')

Saving model checkpoint to /content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/model parameters
Configuration saved in /content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/model parameters/config.json
Model weights saved in /content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/model parameters/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/model parameters/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/model parameters/special_tokens_map.json


In [ ]:
def convert_text_to_tuple_from_csv2(file_path):
      data = []
      df = pd.read_csv(file_path)
      df = df.iloc[34:41,:]
      x = 'Sentence: 1'
      temp = []
      for index, row in df.iterrows():
        flag = row['Sentence #']
        if flag == x:
          temp.append((row["Word"], row["Tag"]))
          # print (index,row['Sentence #'],row["Word"], row["Tag"])
        else:
          x = row['Sentence #']
          data.append(temp)
          temp = [(row["Word"], row["Tag"])]

      return data

test2 =  convert_text_to_tuple_from_csv2('/content/drive/MyDrive/Bracu/THESIS/ShundorBanglaDataset/test2022withPos.csv')

In [ ]:
test2.pop(0)
test2

[[('ওয়াটস', 'B-CW'),
  ('ইওর', 'I-CW'),
  ('রাশি', 'I-CW'),
  ('এর', 'O'),
  ('গান', 'O')]]

In [ ]:
dm3 = NERDataMaker(test2)

['O', 'B-CORP', 'I-CORP', 'B-CW', 'I-CW', 'B-GRP', 'I-GRP', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD']


In [ ]:
test_ds1 = dm3.as_hf_dataset(tokenizer=tokenizer)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
pred = trainer.predict(test_ds1)

***** Running Prediction *****
  Num examples = 1
  Batch size = 64
The following columns in the test set don't have a corresponding argument in `ElectraForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `ElectraForTokenClassification.forward`,  you can safely ignore this message.


In [ ]:
pred


PredictionOutput(predictions=array([[[ 0.73469394, -0.0782921 , -0.85662943,  1.65574   ,
         -0.9004331 ,  1.4212549 , -1.0807363 , -0.02573625,
         -0.23621216,  0.05457312, -0.01600699, -0.05739328,
         -0.84146225],
        [ 8.484081  , -0.7759253 , -2.0861816 ,  2.3474858 ,
         -0.64      ,  0.660538  , -1.3861917 , -0.48701578,
         -2.0857444 , -0.524183  , -2.1525326 , -2.089533  ,
         -2.858199  ],
        [ 8.359815  , -2.0901177 , -1.5861953 ,  1.291045  ,
          0.83380955,  1.0956782 , -0.6960078 , -0.56509733,
         -2.174684  , -1.6424983 , -2.2526214 , -2.0924778 ,
         -2.9714847 ],
        [ 1.3166155 , -1.3436183 , -1.55791   ,  5.497762  ,
          3.1552489 ,  1.9757231 ,  0.45405987, -1.0462646 ,
         -1.4783928 , -1.3382797 , -1.6944964 , -1.8299239 ,
         -2.8547165 ],
        [-1.7547125 , -1.488294  , -1.0517037 ,  2.4751112 ,
          5.73909   ,  2.1706452 ,  2.2597888 , -1.1367117 ,
         -0.7531896 , -1.

In [ ]:
from transformers import pipeline
pipe = pipeline("ner", model=model, tokenizer=tokenizer, device=0) # pass device=0 if using gpu
a = 'ভোজপুর জেলা ওয়েলস'
pipe(a)

[{'entity': 'B-LOC',
  'score': 0.9986279,
  'index': 1,
  'word': 'ভোজ',
  'start': 0,
  'end': 3},
 {'entity': 'I-LOC',
  'score': 0.93395686,
  'index': 2,
  'word': '##পুর',
  'start': 3,
  'end': 6},
 {'entity': 'I-LOC',
  'score': 0.7909466,
  'index': 3,
  'word': 'জেলা',
  'start': 7,
  'end': 11}]

In [ ]:
tokenizer('১৯৮০ গ্রীষ্মকালীন অলিম্পিকে ফ্রান্স কিভাবে পরিমাপ করা হয়')

{'input_ids': [2, 14787, 10081, 3987, 22218, 5996, 2770, 16102, 913, 844, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}